In [28]:
import numpy as np  # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import json         # library to handle JSON files
from bs4 import BeautifulSoup
import requests     # library to handle requests
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import time
import re

In [2]:
# Get Juarez boroughs source 
boroughs_url = 'https://micodigopostal.org/chihuahua/juarez/'
data = requests.get(boroughs_url)
boroughs = BeautifulSoup(data.content, 'lxml')
boroughs = boroughs.find_all('table')[0]

In [3]:
# List the Juarez boroughs

boroughs_list = []
for bor in boroughs.find_all('tbody')[0].find_all('tr'):
    try:
        boroughs_list.append(bor.find_all('td')[0].find_all('a')[0].contents[0])
    except:
        continue


In [4]:
len(boroughs_list)

1071

In [233]:
# Gets Lat and Long from boroughs
def getlongLat(borough):
    try:
        return list(Nominatim(user_agent='my-app').geocode('{},Juarez, Chihuahua'.format(borough)))[-1]
    except:
        return np.nan

In [295]:
df_b = pd.DataFrame(['Minerva', 'Senecú', 'Campestre', 'Nogales', 'Lagos','San Lorenzo', 'Campos Elíseos', 'Campestre', 'Camino Real', 'Bosques del Sol', 'Waterfill', 'Bugambilias', 'Villahermosa', 'Monumental', 'Los Bosques', 'Los Sauces', 'Alameda', 'Américas', 'Antares'])
df_b = df_b.rename(columns={0:"boroughs"})

In [296]:
df_b['lat_long'] = df_b['boroughs'].apply(lambda x: getlongLat(x))

In [297]:
df_b = df_b.dropna()
df_b

,boroughs,lat_long
0,Minerva,"(31.7253053, -106.4345394)"
1,Senecú,"(31.7127726, -106.4051706)"
2,Campestre,"(31.7176118, -106.4114476)"
3,Nogales,"(31.5863277, -106.4822321)"
4,Lagos,"(31.423753, -106.472610911083)"
5,San Lorenzo,"(31.7394918, -106.4249645)"
7,Campestre,"(31.7176118, -106.4114476)"
8,Camino Real,"(31.7137484, -106.4132479)"
12,Villahermosa,"(31.7183447, -106.4321065)"
13,Monumental,"(31.7355212, -106.444169788586)"


In [124]:
#FOURSQUARE API CONNECTION
CLIENT_ID='LEOE4LM4RHRHZYYVCXRTYY5XLUBBKHZMEOAM42KN50ATXWAH'
CLIENT_SECRET='STKIXKKRK3FJ4BNLEJ4E55YZ0A4IMSF35GMVMSCRJAU4E4JN'
VERSION = '20190203' 
RADIUS = 1000 
LIMIT = 100
VENUE = 'bar'

In [125]:
def getVenues(lat, long):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                long, 
                RADIUS, 
                LIMIT)
    return requests.get(url).json()["response"]['groups'][0]['items']

In [298]:
venue_list = []

In [299]:
def setCompetitionVenues(venue):
    if('café' in venue['venue']['name'].lower()):
        popularity = False
        if('is popular' in venue['reasons']['items'][0]['summary'].lower()):
            popularity = True
        return list([venue['venue']['location']['lat'], venue['venue']['location']['lng'], popularity, venue['venue']['name']])

In [300]:
for name, lat_long in zip(df_b['boroughs'], df_b['lat_long']):
    results = getVenues(lat_long[0], lat_long[1])
    for venue in results:
        result = setCompetitionVenues(venue)
        try:
            venue_list.append([name, result[0], result[1], result[2], result[3]])
        except:
            continue

In [301]:
venue_list = pd.DataFrame(venue_list)
venue_list

,0,1,2,3,4
0,Minerva,31.731173,-106.438157,True,Café -Teatro Telón de Arena
1,San Lorenzo,31.738569,-106.426478,True,Juaritoz Café
2,Monumental,31.737472,-106.448619,True,Café Único
3,Monumental,31.731173,-106.438157,True,Café -Teatro Telón de Arena
4,Los Bosques,19.406960,-99.239033,True,Café Punta del Cielo
5,Los Bosques,19.402991,-99.242504,True,Cielito Querido Café
6,Américas,31.737472,-106.448619,True,Café Único
7,Américas,31.748394,-106.448456,True,MAYA café


In [304]:
latitude = 31.7177191
longitude = -106.4434673
jrz_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for name, lat, lng, popular, venue in zip(venue_list[0], venue_list[1], venue_list[2], venue_list[3], venue_list[4]):
    
    color = 'purple'
    
    if(popular):
        color = 'yellow'
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=venue,
        #color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=color,
        fill_opacity=0.7).add_to(jrz_map)


jrz_map